#                                  BANK DATA SET -Intro to Neural Networks | Project

The points distribution for this case is as follows:

1. Read the data set
2. Drop the columns which are unique for all users like IDs (2.5 points)
3. Distinguish the feature and target set (2.5 points)
4. Divide the data set into training and test sets ( 2.5 points)
5. Normalize the train and test data (5 points)
6. Initialize & build the model (10 points)
7. Predict the results using 0.5 as a threshold (5 points)
8. Print the Accuracy score and confusion matrix (2.5 points)

# 1. Loading data file and reading date :-

In [0]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir('/content/drive/My Drive')

In [0]:
bank_df = pd.read_csv("bank.csv")

In [6]:
bank_df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# 2. Drop the columns which are unique for all users like IDs 

In [0]:
bank_df.drop(columns = ['CustomerId','RowNumber'],inplace=True)

In [8]:
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(7), object(3)
memory usage: 937.6+ KB


# Cleaning data


In [0]:
bank_df.drop(columns = ['Surname'],inplace=True)

In [10]:
#Finding the number of counts in each categorical columns:
for i in bank_df.columns:
    if bank_df[i].dtype == 'O':
        print(i,'->',len(bank_df[i].value_counts()))

Geography -> 3
Gender -> 2


In [0]:
#doing manual encoding:
bank_df.replace({'France':0,'Germany':1,'Spain':2,'Male':0,'Female':1},inplace=True)

# 3. Distinguish the feature and target set

In [0]:
from sklearn.model_selection import train_test_split

X = bank_df.drop("Exited", axis=1)
X = X.drop({'Geography','Gender'}, axis=1)
X = X.astype('float32')

In [0]:
y = bank_df[['Exited']]
y = y.astype('float32')

In [0]:
X = np.array(X)
y = np.array(y)

4. Dividing the data into training and test sets: 

In [0]:
train_set, test_set, train_labels, test_labels = train_test_split(X, y, test_size=0.30)

#5. Normalizing the data set: 

In [16]:
%tensorflow_version 2.x
import tensorflow as tf
tf.random.set_seed(42)
tf.__version__

'2.1.0'

In [0]:
from sklearn.preprocessing import Normalizer
import keras
transformer = Normalizer()
train_set = transformer.fit_transform(train_set)

# 6. Building model - 

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(2, activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])


In [27]:
model.fit(train_set, train_labels, 
          validation_data=(test_set, test_labels), 
          epochs=100,
          batch_size=30)

Train on 7000 samples, validate on 3000 samples
Epoch 1/100
7000/7000 [==============================] - 1s 109us/sample - loss: 0.2888 - accuracy: 0.7581 - val_loss: 4823.0866 - val_accuracy: 0.8827
Epoch 2/100
7000/7000 [==============================] - 0s 56us/sample - loss: 0.2875 - accuracy: 0.5731 - val_loss: 4384.5027 - val_accuracy: 0.9200
Epoch 3/100
7000/7000 [==============================] - 0s 58us/sample - loss: 0.2872 - accuracy: 0.5420 - val_loss: 4390.5375 - val_accuracy: 0.9710
Epoch 4/100
7000/7000 [==============================] - 0s 60us/sample - loss: 0.2871 - accuracy: 0.5596 - val_loss: 4499.6392 - val_accuracy: 1.0000
Epoch 5/100
7000/7000 [==============================] - 0s 56us/sample - loss: 0.2870 - accuracy: 0.5877 - val_loss: 4592.5568 - val_accuracy: 1.0000
Epoch 6/100
7000/7000 [==============================] - 0s 59us/sample - loss: 0.2869 - accuracy: 0.6124 - val_loss: 4672.2188 - val_accuracy: 1.0000
Epoch 7/100
7000/7000 [======================

Optimizing model :-

# 7. Predict the results using 0.5 as a threshold:-

In [0]:
w = tf.zeros(shape=(8,1))
b = tf.zeros(shape=(1))

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

In [0]:
def train(x, y_actual, w, b, learning_rate=0.5):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

In [33]:
for i in range(100):
    
    w, b = train(train_set, train_labels, w, b,learning_rate=0.5)
    print('Current Loss on iteration', i, loss(train_labels, prediction(train_set, w, b)).numpy())

Current Loss on iteration 0 0.18864171
Current Loss on iteration 1 0.17785338
Current Loss on iteration 2 0.1714622
Current Loss on iteration 3 0.16767721
Current Loss on iteration 4 0.16543445
Current Loss on iteration 5 0.16410552
Current Loss on iteration 6 0.16331783
Current Loss on iteration 7 0.1628509
Current Loss on iteration 8 0.16257346
Current Loss on iteration 9 0.1624091
Current Loss on iteration 10 0.16231085
Current Loss on iteration 11 0.16225234
Current Loss on iteration 12 0.16221708
Current Loss on iteration 13 0.16219604
Current Loss on iteration 14 0.16218284
Current Loss on iteration 15 0.16217467
Current Loss on iteration 16 0.16216937
Current Loss on iteration 17 0.16216578
Current Loss on iteration 18 0.16216321
Current Loss on iteration 19 0.16216141
Current Loss on iteration 20 0.16215974
Current Loss on iteration 21 0.16215856
Current Loss on iteration 22 0.1621572
Current Loss on iteration 23 0.16215608
Current Loss on iteration 24 0.16215509
Current Loss o

# 8. Print the Accuracy score and confusion matrix:-

In [0]:
from sklearn.metrics import accuracy_score
y_predict = model.predict_classes(test_set, verbose=0)

In [35]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_labels, y_predict)
print('Accuracy: %f' % accuracy)

Accuracy: 0.803667


In [36]:
# precision tp / (tp + fp)
from sklearn.metrics import precision_score
precision = precision_score(test_labels, y_predict)
print('Precision: %f' % precision)

Precision: 0.000000


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
# recall: tp / (tp + fn)
from sklearn.metrics import recall_score
recall = recall_score(test_labels, y_predict)
print('Recall: %f' % recall)

Recall: 0.000000


In [38]:
# f1: 2 tp / (2 tp + fp + fn)
from sklearn.metrics import f1_score
f1 = f1_score(test_labels, y_predict)
print('F1 score: %f' % f1)

F1 score: 0.000000


In [39]:
#confusion matrix:-
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(test_labels, y_predict)
print(matrix)

[[2411    0]
 [ 589    0]]
